In [23]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import re
from matplotlib.ticker import FuncFormatter

# Directories
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / 'data'
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_DIR.mkdir(exist_ok=True)


# --- color maps ---
comp_colors = {
    'Production': '#EF7A15',  # deep orange
    'Operation':  '#35BBCA',  # steel blue
    'EoL':        '#D3DD18',  # soft teal
}
line_colors = {
    'AEF': '#0D0D0D',  # black
    'CEF': '#0D0D0D',  # dark grey
}

# transparency for component stacks
stack_alpha = 0.5

#--- grid spec ---
scenarios  = ['BAU', 'Moderate', 'Aggressive']
rcps       = ['RCP60', 'RCP45', 'RCP26']
comp_cols  = ['N_in_E_prd', 'N_stock_E_opr', 'N_out_E_eol']
comp_names = ['Production', 'Operation', 'EoL']

In [18]:
A = 0.2
A_int = int(round(A * 10))        # 0.2 -> 2
A_str = f"{A_int:02d}"            # -> "02"

In [19]:
# ___ 1. Data Import & Initialise ___

import pandas as pd


csv_path = RESULTS_DIR / f"AEF_CEF_components_A_{A_str}.csv"
aef_cef_components = pd.read_csv(csv_path)

methods = aef_cef_components['method'].unique()
print("Impact categories (methods):")
for m in methods:
    print(f"- {m}")

Impact categories (methods):
- EF v3.1, acidification, accumulated exceedance (AE)
- EF v3.1, climate change, global warming potential (GWP100)
- EF v3.1, climate change: biogenic, global warming potential (GWP100)
- EF v3.1, climate change: fossil, global warming potential (GWP100)
- EF v3.1, climate change: land use and land use change, global warming potential (GWP100)
- EF v3.1, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe)
- EF v3.1, ecotoxicity: freshwater, inorganics, comparative toxic unit for ecosystems (CTUe)
- EF v3.1, ecotoxicity: freshwater, organics, comparative toxic unit for ecosystems (CTUe)
- EF v3.1, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels
- EF v3.1, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P)
- EF v3.1, eutrophication: marine, fraction of nutrients reaching marine end compartment (N)
- EF v3.1, eutrophication: terrestrial, accumulated exceedance (AE)
- EF v

In [20]:
diff_dir = RESULTS_DIR / f"AEF_CEF_components_plot_A_{A_str}"
diff_dir.mkdir(exist_ok=True)      # will hold our CSV export + plots

In [34]:
for method in methods:
    sub = aef_cef_components[aef_cef_components['method']==method]

    # 1) primary y‐limits 0 → 110% max(total)
    total_all = sub[comp_cols].sum(axis=1)
    comp_ylim_global = (0, total_all.max() * 1.1)

    # 2) secondary y‐limits 0 → 110% max(AEF/CEF)
    vals_all = sub[['AEF','CEF']].values.flatten()
    sec_ylim_global = (0, vals_all.max() * 1.1)

    # 3) share‐axis limits (allow extra for <0 and >1)
    all_shares = sub[comp_cols].div(
        sub[comp_cols].sum(axis=1), axis=0
    ).values.flatten()
    share_ylim_global = (
        min(0, all_shares.min()) - 0.1,
        max(1, all_shares.max()) + 0.1
    )

    fig, axes = plt.subplots(3,3,figsize=(15,12), sharex=True)
    for j, sce in enumerate(scenarios):
        axes[0,j].set_title(sce, pad=16, fontsize=14)

    for i, rcp in enumerate(rcps):
      for j, sce in enumerate(scenarios):
        ax   = axes[i,j]
        data = sub[(sub['RCP']==rcp)&(sub['scenario']==sce)].sort_values('year')
        yrs  = data['year'].to_numpy()

        # a) compute totals & signed shares
        comp_vals  = data[comp_cols]
        total_comp = comp_vals.sum(axis=1)
        shares     = comp_vals.div(total_comp, axis=0)

        # b) hidden share‐axis behind ax
        share_ax = ax.twinx()
        share_ax.set_zorder(ax.get_zorder()-1)
        share_ax.patch.set_alpha(0)
        for sp in share_ax.spines.values():
            sp.set_visible(False)
        share_ax.set_ylim(*share_ylim_global)
        share_ax.set_yticks([])

        # c) draw dashed lines at share=0 and share=1
        share_ax.axhline(0, color='grey', linestyle='--', linewidth=1, zorder=0)
        share_ax.axhline(1, color='grey', linestyle='--', linewidth=1, zorder=0)

        # d) stack positives above zero
        bottom_pos = np.zeros(len(yrs))
        for col, name in zip(comp_cols, comp_names):
            pos = np.clip(shares[col].to_numpy(), 0, None)
            share_ax.bar(
                yrs, pos, bottom=bottom_pos, width=0.8,
                color=comp_colors[name], alpha=stack_alpha,
                align='center'
            )
            bottom_pos += pos

        # e) stack negatives below zero
        bottom_neg = np.zeros(len(yrs))
        for col, name in zip(comp_cols, comp_names):
            neg = np.clip(shares[col].to_numpy(), None, 0)
            share_ax.bar(
                yrs, neg, bottom=bottom_neg, width=0.8,
                color=comp_colors[name], alpha=stack_alpha,
                align='center'
            )
            bottom_neg += neg

        # f) bring main ax forward & plot total line
        ax.set_zorder(share_ax.get_zorder()+1)
        ax.patch.set_visible(False)
        ax.set_ylim(*comp_ylim_global)
        if j==0:
            ax.set_ylabel('Total burdens')
        else:
            ax.yaxis.set_visible(False)
        
        for yr in (2035, 2036, 2050):
            ax.axvline(yr, color='grey', linestyle='--', linewidth=1, zorder=0)

        total_color = '#8B0000'
        ax.plot(yrs, total_comp, color=total_color, linewidth=2)
        total_handle = Line2D([0],[0], color=total_color, lw=2, label='Total')

        # g) true secondary axis for AEF/CEF
        ax2 = ax.twinx()
        ax2.set_zorder(ax.get_zorder()+1)
        ax2.set_ylim(*sec_ylim_global)
        ax2.plot(yrs, data['AEF'], color=line_colors['AEF'], linestyle='-',  linewidth=1)
        ax2.plot(yrs, data['CEF'], color=line_colors['CEF'], linestyle='--',linewidth=1)
        if j==2:
            ax2.set_ylabel('AEF / CEF')
        else:
            ax2.yaxis.set_visible(False)

    # legend & save (unchanged)
    comp_patches = [Patch(color=comp_colors[n], label=n) for n in comp_names]
    line_handles = [
        Line2D([0],[0], color=line_colors['AEF'], linestyle='-', lw=2, label='AEF'),
        Line2D([0],[0], color=line_colors['CEF'], linestyle='--', lw=2, label='CEF')
    ]
    fig.legend(
        handles=comp_patches + [total_handle] + line_handles,
        loc='lower center', ncol=6, frameon=False,
        fontsize=12, title='Burden Share & AEF/CEF'
    )
    fig.tight_layout(rect=[0,0.03,1,1])
    out_png = diff_dir / f"{re.sub(r'[^0-9A-Za-z]+','_',method)}_components_A_{A_str}.png"
    fig.savefig(out_png, dpi=150, bbox_inches='tight')
    plt.close(fig)
    print(f"▶ Saved: {out_png}")

print("All plots saved.")


▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_acidification_accumulated_exceedance_AE__components_A_02.png
▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_climate_change_global_warming_potential_GWP100__components_A_02.png
▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_climate_change_biogenic_global_warming_potential_GWP100__components_A_02.png
▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_climate_change_fossil_global_warming_potential_GWP100__components_A_02.png
▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_climate_change_land_use_and_land_use_change_global_warming_potential_GWP100__components_A_02.png
▶ Saved: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\AEF_CEF_components_plot_A_02\EF_v3_1_ecotoxicity_freshwater_comparative_toxic_unit_for_ecosystems_CTUe__components_A_02.png
▶ Saved

In [42]:

# --- 2. Selected combination “baseline” plot & save ---
baseline_dir = RESULTS_DIR / f"Baseline_plot_A_{A_str}"
baseline_dir.mkdir(exist_ok=True)

# pick your one combination:
selected_method   = "EF v3.1, particulate matter formation, impact on human health"
selected_rcp      = "RCP60"
selected_scenario = "BAU"

# filter down to just that slice
sub = aef_cef_components[aef_cef_components['method'] == selected_method]
data = (
    sub[(sub['RCP'] == selected_rcp) & (sub['scenario'] == selected_scenario)]
    .sort_values('year')
)

yrs        = data['year'].to_numpy()
comp_vals  = data[comp_cols]
total_comp = comp_vals.sum(axis=1)
shares     = comp_vals.div(total_comp, axis=0)

# compute limits exactly as before
# primary axis: 0 → 110% max(total)
comp_ylim = (0, total_comp.max() * 1.1)
# secondary axis: 0 → 110% max(AEF,CEF)
vals = data[['AEF','CEF']].values.flatten()
sec_ylim  = (0, vals.max() * 1.1)
# share axis: allow a bit beyond [0,1]
all_shares = shares.values.flatten()
share_ylim = (min(0, all_shares.min()) - 0.1,
              max(1, all_shares.max()) + 0.1)

# build a single‐panel figure
fig, ax = plt.subplots(figsize=(8,6))

# 1) hidden share‐axis behind ax for stacked bars
share_ax = ax.twinx()
share_ax.set_zorder(ax.get_zorder()-1)
share_ax.patch.set_alpha(0)
for sp in share_ax.spines.values():
    sp.set_visible(False)
share_ax.set_ylim(*share_ylim)
share_ax.set_yticks([])

# dashed guides at share=0 and share=1
share_ax.axhline(0, color='grey', linestyle='--', linewidth=1, zorder=0)
share_ax.axhline(1, color='grey', linestyle='--', linewidth=1, zorder=0)

# stacked positives above 0
bottom_pos = np.zeros_like(yrs, dtype=float)
for col, name in zip(comp_cols, comp_names):
    pos = np.clip(shares[col].to_numpy(), 0, None)
    share_ax.bar(
        yrs, pos, bottom=bottom_pos, width=0.8,
        color=comp_colors[name], alpha=stack_alpha,
        align='center'
    )
    bottom_pos += pos

# stacked negatives below 0
bottom_neg = np.zeros_like(yrs, dtype=float)
for col, name in zip(comp_cols, comp_names):
    neg = np.clip(shares[col].to_numpy(), None, 0)
    share_ax.bar(
        yrs, neg, bottom=bottom_neg, width=0.8,
        color=comp_colors[name], alpha=stack_alpha,
        align='center'
    )
    bottom_neg += neg

# 2) bring main ax forward & plot total burdens
ax.set_zorder(share_ax.get_zorder()+1)
ax.patch.set_visible(False)
ax.set_ylim(*comp_ylim)
ax.set_ylabel('Total burdens')

for yr in (2035, 2036, 2050):
    ax.axvline(yr, color='grey', linestyle='--', linewidth=1, zorder=0)

ax.plot(yrs, total_comp, color='#8B0000', linewidth=2)
total_handle = Line2D([0],[0], color='#8B0000', lw=2, label='Total')

# 3) true secondary axis for AEF/CEF
ax2 = ax.twinx()
ax2.set_zorder(ax.get_zorder()+1)
ax2.set_ylim(*sec_ylim)
ax2.plot(yrs, data['AEF'], color=line_colors['AEF'], linestyle='-',  linewidth=1)
ax2.plot(yrs, data['CEF'], color=line_colors['CEF'], linestyle='--',linewidth=1)
ax2.set_ylabel('AEF / CEF')

# 4) legend & save
comp_patches = [Patch(color=comp_colors[n], label=n) for n in comp_names]
line_handles = [
    Line2D([0],[0], color=line_colors['AEF'], linestyle='-', lw=2, label='AEF'),
    Line2D([0],[0], color=line_colors['CEF'], linestyle='--', lw=2, label='CEF')
]
fig.legend(
    handles=comp_patches + [total_handle] + line_handles,
    loc='lower center', ncol=6, frameon=False,
    fontsize=12
)

fig.tight_layout(rect=[0, 0.05, 1, 1])
out_png = baseline_dir / f"{re.sub(r'[^0-9A-Za-z]+','_',selected_method)}_Baseline_components_A_{A_str}.png"
fig.savefig(out_png, dpi=150, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

print(f"▶ Baseline plot saved to: {out_png}")


▶ Baseline plot saved to: E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\Baseline_plot_A_02\EF_v3_1_particulate_matter_formation_impact_on_human_health_Baseline_components_A_02.png
